In [ ]:
# imports
import torch
from torch import nn

import torchvision
import torchvision.transforms as transforms

from torch.optim import lr_scheduler

from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder

from momentumnet import MomentumNet
from momentumnet import transform_to_momentumnet

import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed
from joblib.externals.loky.backend.context import get_context

import time
import copy
import random
import pickle
import tarfile

import cv2

from memory_profiler import memory_usage

from smallnorb.dataset import SmallNORBDataset

# Resolve some errors
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Define Datasets
For the experiments with the Evolutionary Stochastic Gradient Descent algorithm we use the CIFAR10 and the smallNORB dataset. Since smallNORB is not available via torchvision we used python wrapper by ndrplz, more informations under https://github.com/ndrplz/small_norb

In [ ]:
def create_CIFAR10_loader(batch_size=100):
    transform_train = transforms.Compose([
        transforms.Pad(4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    # CIFAR-10 dataset
    train_dataset = torchvision.datasets.CIFAR10(root='data/', train=True, transform=transform_train, download=True)
    test_dataset = torchvision.datasets.CIFAR10(root='data/', train=False, transform=transform_test)

    # Define DataLoaders
    # use 'loky' to work with joblib
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,  num_workers=4, multiprocessing_context=get_context('loky'))
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,   batch_size=batch_size, shuffle=False, num_workers=4, multiprocessing_context=get_context('loky'))
    
    return train_loader, test_loader

def create_SmallNORB_dataset(batch_size=100):
    dataset = SmallNORBDataset(dataset_root='data')
    
    # preprocess dataset
    # -> convert from grayscale to RGB to get 3 channels
    # -> move channel axis to the first position
    # -> convert category from byte to long
    train_dataset = []
    for d in dataset.data['train']:
        z = (np.array(np.moveaxis(cv2.cvtColor(d.image_lt, cv2.COLOR_GRAY2RGB), -1, 0), dtype=np.float32), np.array(d.category, dtype=np.int64))
        train_dataset.append(z)
    
    test_dataset = []
    for d in dataset.data['test']:
        z = (np.array(np.moveaxis(cv2.cvtColor(d.image_lt, cv2.COLOR_GRAY2RGB), -1, 0), dtype=np.float32), np.array(d.category, dtype=np.int64))
        test_dataset.append(z)
    
    # Define DataLoaders
    # use 'loky' to work with joblib
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=3, multiprocessing_context=get_context('loky'))
    test_loader  = torch.utils.data.DataLoader(dataset=test_dataset,  batch_size=batch_size, shuffle=False, num_workers=3, multiprocessing_context=get_context('loky'))
    
    return train_loader, test_loader

# Res-Net
We will now define the ResNets we are going to use for the experiments. The implementation of these networks was created by akamaster, more informations under https://github.com/akamaster/pytorch_resnet_cifar10/blob/master/resnet.py.



In [ ]:
import torch.nn.functional as F
import torch.nn.init as init

def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
              
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):        
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)

        self.fc = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def resnet20(num_classes=10):
    return ResNet(BasicBlock, [3, 3, 3], num_classes=num_classes)


def resnet32(num_classes=10):
    return ResNet(BasicBlock, [5, 5, 5], num_classes=num_classes)


def resnet44(num_classes=10):
    return ResNet(BasicBlock, [7, 7, 7], num_classes=num_classes)


def resnet56(num_classes=10):
    return ResNet(BasicBlock, [9, 9, 9], num_classes=num_classes)


def resnet110(num_classes=10):
    return ResNet(BasicBlock, [18, 18, 18], num_classes=num_classes)


def resnet1202(num_classes=10):
    return ResNet(BasicBlock, [200, 200, 200], num_classes=num_classes)

Additionally we will define a function to convert the ResNets above into momentum ResNets

In [ ]:
def create_momentum_resnet(resnet, use_bp=False, gam=0.9):
    """
    A function to create a momentum resnet with randomly initialized weights.
    
    Parameters:
    resnet: The network to be converted into a momentum ResNet
    use_bp: If False it will use the reversible architecture to calculate gradients, else it will use normal backpropagation
    gam:    Determines the gamma value in the momentum term.      
    """
    
    mresnet = transform_to_momentumnet(resnet,
                                       ["layer1", "layer2", "layer3"], # The layers to make invertible
                                       gamma=gam,                      # Momentum term of the momentum res-net
                                       use_backprop=use_bp,            # False to have smaller memory footprint
                                       is_residual=True)               # True, because forward rule is x + f(x)
    
    return mresnet

Also define some auxilliary functions for saving, backups and printing weights.

In [ ]:
def print_weights(network):
    for param in network.parameters():
        print(param.data)
        
    print(f"-------------------")
    
def save_to_file(network, name):
    torch.save(network.state_dict(), f"{name}.pth")
    
def load_from_file(name):
    resnet = create_momentum_resnet()
    resnet.load_state_dict(torch.load(f"backups/{name}.pth"))
    return resnet

def create_backup(population, k):
    """
    Creates backups of the population in the 'backups' directory.
    """
    
    directory_exists = os.path.exists("backups/")
    if not directory_exists:
        os.makedirs("backups/")
    
    # create reminder of what generation we finished
    filename = f"backups/!finished_gen_{k}.txt"
    f = open(filename, 'w')
    f.close()
    
    # save population
    i = 0
    for network in population:
        save_to_file(network, f"backups/backup_network_{i}")
        i = i + 1

# Stochastic Gradient Descent
The following cells will implement the SGD part of the Evolutionary Stochastic Gradient Descent algorithm. To make it more stable we added weight decay and gradient clipping.

In [ ]:
def SGD_training(network, SGD_steps, lr, momentum, data_loader):
    """ 
    Performs SGD training for the given network.
    
    In order to avoid overloading the GPU with too many networks to train, we will send the 
    network to the GPU for the training process, afterwards we will send it back to the CPU.
    Also the optimizer and the criterion are defined here, so no need to define them earlier.
    
    Parameters:
    network:     the network to be trained using SGD
    SGD_steps:   the number of SGD steps we will perform
    lr:          the base learning rate
    momentum:    the momentum for SGD
    data_loader: the data loader to load the training examples
    
    """
    network.to(device)
    network.train()
    
    lr_multiplier = random.uniform(0.9, 1.1)
    
    # Create optimizer and criterion
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(network.parameters(), lr=lr_multiplier*lr, momentum=momentum, nesterov=True, weight_decay=0.0001)
    
    total_step = len(data_loader)
    
    for s in range(0, SGD_steps):        
        for i, (images, labels) in enumerate(data_loader):    
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = network(images)            
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            
            #Gradient Value Clipping
            nn.utils.clip_grad_value_(network.parameters(), clip_value=1.0)
            
            optimizer.step()
            
            #### print a debug message about status of training ####
            #if (i+1) % 100 == 0:
            #    print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}".format(s+1, SGD_steps, i+1, total_step, loss.item()))
        
            # make some clean-up
            del loss, outputs
    
    # move network back to cpu and return
    network.cpu()
    return network

# Genetic Algorithm
Now we will implement the Genetic Algorithms part. For Selection we chose Roulette-Wheel-Selection, Crossover is done through Average-Crossover and for Mutation we use a zero-meaned normal distribution.

In [ ]:
def crossover_and_mutation(parents, sigma=0.01):
    """
    Performs crossover and mutation in order to create a new offspring.
    
    The offspring is created using the stated dicts of the parents. First we sum up the 
    weights and biases of the parents and then we take an average of them. We also add 
    noise in form of a Normaldistribution (mutation given as: N(0, sigma^2)).
    
    Parameters:
    parents: The list of parents from which the offspring is created.
    sigma:   The mutation strength of the normal distribution
    
    Returns: The newly created offspring
    """
    
    base_sd = parents[0].state_dict()
    
    # adjust keys to determine which layers to should be affected by crossover and mutation
    # Example:
    #keys = ['fc.weight', 'fc.bias']
    keys = base_sd                    # use all layers to be affected
    
    # sum of the weights of the parent
    for i in range(1, len(parents)):
        # get state dict of other parent
        parent_sd = parents[i].state_dict()
        
        # add parent weights to the base state dict
        for key in keys:
            base_sd[key] = base_sd[key] + parent_sd[key]
            
    
    # average the sum
    num_parents = len(parents)
    for key in keys:
        
        # Create mutation
        tensor_size = base_sd[key].size()
        random_tensor = torch.normal(mean=0.0, std=sigma, size=tensor_size)
        
        # Average and add mutation
        base_sd[key] = base_sd[key] / num_parents + random_tensor
    
    # create offspring
    offspring = create_momentum_resnet()
    offspring.load_state_dict(base_sd)
    return offspring
    

def create_offspring(population, fitness, rho, sigma):
    """
    Creates new offspring based on selection, crossover and mutation.
    
    Parameters:
    population: The population of the generation
    fitness:    The fitness evalutaions of the individuals in the population
    rho:        How many parents we select for crossover
    
    Returns: A newly created offspring, produced by selection, crossover and mutation
    """
    
    # Perform selection
    parents = random.choices(population, weights=fitness, k=rho)
    
    # Perform crossover and mutation
    offspring = crossover_and_mutation(parents, sigma)
    return offspring


def GA_training(population, pop_size, offspring_size, elitist_level, rho, sigma, data_loader):
    """
    Performs GA to create a new population for the next generation
    
    Parameters:
    population:     The population of the curret generation on which we will create new individuals
    pop_size:       The population size of the whole generation
    offspring_size: The total number of offsprings the GA-step will generate
    elitist_level:  The percentage of elite individuals which get carried over to next generation
    data_loader:    The data loader on which fitness will be evaluated 
    
    Returns: The new population for the next generation
    """
    
    ### Calculate fitness of trained population ###
    # Parallel version:
    #fitness = Parallel(n_jobs=2)(delayed(calc_loss)(population[i], data_loader) for i in range(pop_size))
    
    fitness = [calc_loss(population[i], data_loader) for i in range(pop_size)]
    print(f"--- -- Finished fitness evaluation, length: {len(fitness)}")
    
    ### Create offspring population ###
    fitness_weighted = [1/f for f in fitness]   # take inverse of loss so lower losses get higher fitness-values
    offspring_population = [create_offspring(population, fitness_weighted, rho, sigma) for i in range(offspring_size)]
    print(f"--- -- Finished creating offspring population")
    
    ### Evaluate fitness of offsprings ###
    # Parallel version:
    #offspring_fitness = Parallel(n_jobs=2)(delayed(calc_loss)(offspring_population[i], data_loader) for i in range(offspring_size))
    
    offspring_fitness = [calc_loss(offspring_population[i], data_loader) for i in range(offspring_size)]
    print(f"--- -- Finished evaluating fitness of offspring population")
    
    # Combine fitness and population lists
    combined_fitness = fitness + offspring_fitness
    combined_population = population + offspring_population
    
    # sort population by their fitness values
    sorted_population = [pop for _, pop in sorted(zip(combined_fitness, combined_population), key=lambda pair: pair[0])]
    sorted_fitness = [loss for loss, _ in sorted(zip(combined_fitness, combined_population), key=lambda pair: pair[0])]
    
    # Select m-elitists from sorted population
    m = int(pop_size * elitist_level)
    new_population = sorted_population[0:m]
    
    # Fill up rest of population
    difference = pop_size - m
    remaining_population = list(set(sorted_population) - set(new_population))
    filler_population = random.sample(remaining_population, difference)
    
    # assemble new population and return
    new_population = new_population + filler_population
    return new_population, sorted_fitness

Define some possible fitness evaluation functions. In our experiments the fitness was defined through the loss value on the training examples -> the lower the loss, the higher the fitness was. Another possibility would be to use the accuracy on the test examples. NOTE: The GA part above has to be adjusted when chaning the fitness functions.

In [ ]:
# Test the model
def test_model(network, data_loader):
    """
    Tests the given network on the provided DataLoader.
    
    Parameters:
    network:     The network to test 
    data_loader: The data to test on
    
    Returns: The accuracy of the network on the dataset
    """
    
    # init accuracy
    accuracy = 0.0
    
    network.to(device)
    network.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = network(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        #print('Accuracy of the model on the test images: {} %'.format(accuracy))
        
    # send network back to cpu
    network.cpu()
    return accuracy

def calc_loss(network, data_loader):
    """
    Calculates the loss of a network on the given DataLoader and returns the loss.
    
    Parameters:
    network:     The network to test
    data_loader: The data to test the network on
    
    Returns: The loss of the network in the dataset
    """
    
    network.to(device)
    criterion = nn.CrossEntropyLoss()
    running_loss = 0.0
    
    for i, (images, labels) in enumerate(data_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = network(images)
        loss = criterion(outputs, labels)
        
        running_loss += float(loss.item() * images.size(0))
        del loss, outputs
        
    network.cpu()
    return float(running_loss / len(data_loader.sampler))

# Evolutionary Stochastic Gradient Descent
The following cells will combine the SGD part with the GA part to create the hybrid genetic algorithm

In [ ]:
def ESGD_algorithm(population, max_generations, SGD_steps, GA_steps, offspring_size, elitist_level, rho, base_lr, lr_milestones, train_loader, test_loader):
    """
    Performs the Evolutionary Stochastic Gradient Descent algorithm for training neural networks.
    
    The hybrid genetic algorithm is a mixture between Genetic Algorithms (GA) and Stochastic Gradient
    Descent (SGD). We first start off by initalizing our population of networks. Then we switch back
    and forth between performing SGD and GA.
    
    Parameters:
    population:     The initial populatoin
    max_generatios: How many generations the whole algorithm should run
    SGD_steps:      How many SGD steps we perform per generation
    GA_steps:       How many GA steps we perform per generation (usually just one)
    offspring_size: How many offsprings to generate during GA
    elitist_level:  Number of elites getting a fix place in next generation
    rho:            Number of parents to get draw for producing offspring (usually 2)
    base_lr:        The base learning rate
    lr_milestones:  The milestones on which the learning rate changes -> Usage: (GENERATION_NUMBER, NEW_LERANING_RATE)
    train_loader:   Data Loader for loading training examples
    test_loader:    Data Loader for loading test examples (for fitness evaluation)
    
    Returns: The trained population
    """
    
    # Create initial population
    pop_size = len(population)
    print(f"Starting with population of size: {pop_size}")
    
    # create a checkpoint after before starting training to test backups
    create_backup(population, 0)
    
    learning_rate = base_lr
    
    for k in range(max_generations):
        print(f"Currently in generation {k+1}")
        
        # Adjust learning rate
        for (mile_k, mile_lr) in lr_milestones:
            if mile_k == (k+1):
                learning_rate = mile_lr
                print(f"Changed learning rate to {learning_rate}")
        
        # Perform SGD
        print(f"--- Starting SGD")
        
        # Parallel version
        #population = Parallel(n_jobs=2)(delayed(SGD_training)(population[i], SGD_steps, learning_rate, 0.9, train_loader) for i in range(len(population)))
        
        # Sequential version
        population = [SGD_training(population[i], SGD_steps, learning_rate, 0.9, train_loader) for i in range(pop_size)]
        
        print(f"--- Finished SGD")
        
        # create a checkpoint after SGD-steps
        create_backup(population, k)
            
        # Perform GA
        print(f"--- Starting GA")
        sorted_fitness = []          # store the sorted fitness values to maybe use in data collection
        for i in range(0, GA_steps):
            sigma = 0.01 / (k+1)
            population, sorted_fitness = GA_training(population, pop_size, offspring_size, elitist_level, rho, sigma, train_loader)
        print(f"--- Finished GA")
        
        # create a checkpoint after generation finished
        create_backup(population, k)
        
    print(f"Finished training process")
    return population

# Start training process
We have now defined the whole training algorithm. The next step is to actually perform training.

In [ ]:
# Get the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.get_device_name(0))

In [ ]:
# Define training dataset
train_loader, test_loader = create_CIFAR10_loader(batch_size=64)
#train_loader, test_loader = create_SmallNORB_dataset(batch_size=64)

In [ ]:
# Create population and start training process
population = [resnet20() for i in range(128)]
#population = [create_momentum_resnet(resnet20()) for i in range(128)]

trained_population = ESGD_algorithm(population=population,
                                    max_generations=8,
                                    SGD_steps=20,
                                    GA_steps=1,
                                    offspring_size=768,
                                    elitist_level=0.6,
                                    rho=2,
                                    base_lr=0.1,
                                    lr_milestones = [(4, 0.01), (6, 0.001)],
                                    train_loader=train_loader,
                                    test_loader=test_loader
                                    )

# Memory Consumption
In this section we now want to measure the memory requirements of both network verions

In [ ]:
# Define dataset
#train_loader, _ = create_SmallNORB_dataset(batch_size=256)
train_loader, _ = create_CIFAR10_loader(batch_size=256)

#### Memory requirements of regular ResNets

In [ ]:
res_networks = [create_momentum_resnet(resnet20(),   use_backprop=True, gamma=0.0),
                create_momentum_resnet(resnet32(),   use_backprop=True, gamma=0.0),
                create_momentum_resnet(resnet44(),   use_backprop=True, gamma=0.0),
                create_momentum_resnet(resnet56(),   use_backprop=True, gamma=0.0),
                create_momentum_resnet(resnet110(),  use_backprop=True, gamma=0.0),
                create_momentum_resnet(resnet1202(), use_backprop=True, gamma=0.0)
               ]

criterion = nn.CrossEntropyLoss()

def train(net):
    loss = criterion(net(images), labels)
    loss.backward

print("Starting memory measurments")
mem_res = []
for network in res_networks:
    (images, labels) = next(iter(train_loader))
    used_mem_res = np.max(memory_usage((train, (network,))))
    mem_res.append(used_mem_res)
    
print(mem_res)

#### Memory requirements of momentum ResNets

In [ ]:
mom_networks = [create_momentum_resnet(resnet20(),   use_backprop=False, gamma=0.9),
                create_momentum_resnet(resnet32(),   use_backprop=False, gamma=0.9),
                create_momentum_resnet(resnet44(),   use_backprop=False, gamma=0.9),
                create_momentum_resnet(resnet56(),   use_backprop=False, gamma=0.9),
                create_momentum_resnet(resnet110(),  use_backprop=False, gamma=0.9),
                create_momentum_resnet(resnet1202(), use_backprop=False, gamma=0.9)
               ]

criterion = nn.CrossEntropyLoss()

def train(net):
    loss = criterion(net(images), labels)
    loss.backward
    
print("Starting memory measurments")
mem_mom = []
for network in mom_networks:
    (images, labels) = next(iter(train_loader))
    used_mem_res = np.max(memory_usage((train, (network,))))
    mem_mom.append(used_mem_res)
    
print(mem_mom)

In [ ]:
# plot results
x_labels = ["ResNet20", "ResNet32", "ResNet44", "ResNet56", "ResNet110", "ResNet1202"]

end   = len(x_labels)
start = 0

plt.plot(x_labels[start:end], mem_res[start:end], label="Regular ResNet")
plt.plot(x_labels[start:end], mem_mom[start:end], label="Momentum ResNets")
plt.ylabel("Memory (Gib)")
plt.legend()